# Sample project: stock prediction and trading

What question you are looking to answer?
Why does this question matter?
What data did you use?
Where you got the data?
How was the data sampled?
How was the data obtained?
How did you explore the data?
How did you model the data?
Why you chose to model it that way?
What code did you write / use?
How did you fit the model?
How did you validated the model?
How you know the results make sense?
How did you visualized the results?
How you would communicate the results to others?
What did you learn?
What you would do differently if you did this project again?
If you were going to continue this work, what next steps you would take with this project?
How you would explain what you did to a data scientist?
How you would explain what you did to a non-data scientist?


## Setup

Configure logging, import config, and connect to database.

In [1]:
%matplotlib inline

import os
import time
import datetime
import logging

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import config
from data.database import Database
from data.api_manager import API_Manager
from utils.descriptive_stats import weighted_mean

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%H:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 20)

db = Database(config.database)
api = API_Manager(config.api_key)


## Fetch trades from API

Fetch all trades for ticker within date range and store them in the database.

In [2]:
ticker = 'MSFT'
date_from = '2015-01-01'
date_to = '2020-12-31'

ticker_details = db.get_ticker_details(ticker)
if ticker_details is None:
    ticker_details = db.store_ticker_details(api.get_ticker_details(ticker))

dates_with_trades = db.get_open_dates(ticker_details['exchange'], date_from, date_to)
dates_already_stored = db.get_stored_dates('trades', ticker)
dates_to_fetch = [d for d in dates_with_trades if d not in dates_already_stored]

logging.info(f'Fetching {len(dates_to_fetch)} days of {ticker} trades.')
for date in dates_to_fetch:
    
    # Download raw trades.
    time_before_fetch = time.time()
    trades = api.get_daily_trades(ticker, date)
    
    # Convert data to dataframe.
    time_before_process = time.time()
    trades = pd.DataFrame(trades)[['t', 'p', 's']]
    trades = trades.rename(columns={'t': 'timestamp', 'p': 'price', 's': 'volume'})

    # Store raw trades and bars in database.
    time_before_store = time.time()
    db.store_trades(ticker, date, trades)
    
    time_to_fetch = int(round(time_before_process - time_before_fetch))
    time_to_process = int(round(time_before_store - time_before_process))
    time_to_store = int(round(time.time() - time_before_store))
    logging.info(
        f'{ticker} {date} - '
        f'fetch time: {time_to_fetch}s, '
        f'process time: {time_to_process}s, '
        f'store time: {time_to_store}s'
    )

21:13:50 Fetching 0 days of MSFT trades.


## Feature engineering

For every `n` seconds during open hours, calculate target price and features. For now, `n` is set to `1`. Features are stored back in the database as they can take a while to calculate. 

Feature ideas:

- Summary stats of previous prices and volumes (https://alphascientist.com/feature_engineering.html).
- Trading markers (https://blog.roboforex.com/blog/2020/01/10/creating-a-trading-strategies-based-on-the-mean-reversion-and-momentum/).
- Stats on quotes.
- Relevant news articles.
- Mentions on social networks.


In [3]:

def stats_per_second(trades, time_from, time_to):
    """ Calculate descriptive stats for trades for every second.
    
    Stats are calculated in the second _before_ the time listed to prevent
    future data leakage.
    
    """
    
    time_period = pd.date_range(time_from, time_to, freq='1S')
    
    grouper = pd.Grouper(key='time', freq='1S')
    trades_grouped = trades.groupby(grouper)
    
    price_stats = trades_grouped['price'].agg(
        ['count', 'mean', 'median', 'min', 'max', 'std']
    )
    price_mean_weighted = weighted_mean(trades, 'price', 'volume', grouper)
    
    volume_stats = trades_grouped['volume'].agg(
        ['sum', 'mean', 'median', 'min', 'max', 'std']
    )
    
    
    stats = pd.DataFrame(index=time_period) \
        .join(price_mean_weighted, how='left') \
        .join(price_stats.add_prefix('price_'), how='left') \
        .join(volume_stats.add_prefix('volume_'), how='left') \
    
    return stats

ticker = 'MSFT'
exchange = db.get_ticker_details(ticker)['exchange']

stored_dates = sorted(db.get_stored_dates('trades', ticker))
open_hours = db.get_open_hours(stored_dates, exchange)

for date in [stored_dates[4], stored_dates[100]]:
    
    trades = db.get_trades(ticker, date)
    print(trades.iloc[0]['time'], trades.iloc[-1]['time'])
    
    open_time, close_time = open_hours[date]
    
    prices = stats_per_second(
        trades, 
        datetime.datetime.combine(date, open_time), 
        datetime.datetime.combine(date, close_time)
    )


#     prices = trades.groupby(period_grouper)['price'].median()
#     # Propegate last valid observation forward.
#     prices = prices.fillna(method='ffill')

prices.iloc[5000:].head(20).fillna(method='ffill')
# prices[prices> 0]

2015-01-08 04:00:00.106000 2015-01-08 19:59:00.306000
2015-05-28 04:20:10.281000 2015-05-28 19:20:17.191000


,weighted_average,price_count,price_mean,price_median,price_min,price_max,price_std,volume_sum,volume_mean,volume_median,volume_min,volume_max,volume_std
2015-05-28 10:23:20,0.000000,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2015-05-28 10:23:21,0.000000,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2015-05-28 10:23:22,47.729193,12,47.729233,47.7300,47.7200,47.7400,0.009013,2446,203.833333,150.0,100.0,500.0,145.678247
2015-05-28 10:23:23,47.732112,12,47.720825,47.7200,47.7150,47.7399,0.006309,3187,265.583333,100.0,100.0,1990.0,543.765902
2015-05-28 10:23:24,0.000000,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2015-05-28 10:23:25,47.716509,31,47.716129,47.7150,47.7150,47.7200,0.002125,5382,173.612903,146.0,1.0,562.0,131.880420
2015-05-28 10:23:26,47.715000,2,47.715000,47.7150,47.7150,47.7150,0.000000,300,150.000000,150.0,100.0,200.0,70.710678
2015-05-28 10:23:27,0.000000,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2015-05-28 10:23:28,47.715000,1,47.715000,47.7150,47.7150,47.7150,NaN,10,10.000000,10.0,10.0,10.0,NaN
2015-05-28 10:23:29,0.000000,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [4]:
prices.iloc[5000:].head(20).fillna(method='ffill')

,weighted_average,price_count,price_mean,price_median,price_min,price_max,price_std,volume_sum,volume_mean,volume_median,volume_min,volume_max,volume_std
2015-05-28 10:23:20,0.000000,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2015-05-28 10:23:21,0.000000,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2015-05-28 10:23:22,47.729193,12,47.729233,47.7300,47.7200,47.7400,0.009013,2446,203.833333,150.0,100.0,500.0,145.678247
2015-05-28 10:23:23,47.732112,12,47.720825,47.7200,47.7150,47.7399,0.006309,3187,265.583333,100.0,100.0,1990.0,543.765902
2015-05-28 10:23:24,0.000000,0,47.720825,47.7200,47.7150,47.7399,0.006309,0,265.583333,100.0,100.0,1990.0,543.765902
2015-05-28 10:23:25,47.716509,31,47.716129,47.7150,47.7150,47.7200,0.002125,5382,173.612903,146.0,1.0,562.0,131.880420
2015-05-28 10:23:26,47.715000,2,47.715000,47.7150,47.7150,47.7150,0.000000,300,150.000000,150.0,100.0,200.0,70.710678
2015-05-28 10:23:27,0.000000,0,47.715000,47.7150,47.7150,47.7150,0.000000,0,150.000000,150.0,100.0,200.0,70.710678
2015-05-28 10:23:28,47.715000,1,47.715000,47.7150,47.7150,47.7150,0.000000,10,10.000000,10.0,10.0,10.0,70.710678
2015-05-28 10:23:29,0.000000,0,47.715000,47.7150,47.7150,47.7150,0.000000,0,10.000000,10.0,10.0,10.0,70.710678


### Target variable
The future price to predict. The variable classified into either `buy`, `don't sell`, or `sell` depending on the relative increase from the current price to the price in `m` seconds. For now, `m` is set to `5`. 

In [ ]:
ticker = 'MSFT'
exchange = db.get_ticker_details(ticker)['exchange']

frequency = '1S' # 1 second

stored_dates = sorted(db.get_stored_dates('trades', ticker))


trades

for date in stored_dates[::-1]:
    
    # Determine open hours (by Robinhood/Alpaca)

    trades = pd.DataFrame(
        db.get_trades(ticker, date), 
        columns=['timestamp', 'price', 'volume']
    )
    trades['time'] = pd.to_datetime(trades['timestamp']) - pd.DateOffset(hours=5) # America/New Yow
    
    price_index = pd.date_range(
        datetime.datetime.combine(date, start_time),
        datetime.datetime.combine(date, close_time),
        freq=frequency,
        closed='left'
    )
    
    prices = pd.DataFrame(index=price_index).merge(
        trades.groupby(pd.Grouper(key='time', freq=frequency))['price'].median(),
        how='left',
        left_index=True,
        right_index=True
    )
    
    # Propegate last valid observation forward.
    prices = prices.fillna(method='ffill')

    break
    
prices

description = 'Price'
db.store_feature(ticker, 'target', prices, description)

### Check features for outliers

### Check feature importance

## Hyperparameter tuning

- Feature time window
- Target variable future time